In [4]:
import fontTools.ttLib

def has_arabic_glyphs(font_path, text):
    """
    Checks if a font has glyphs for the given Arabic text.

    Args:
        font_path (str): Path to the font file.
        text (str): Arabic text to check.

    Returns:
        bool: True if the font has glyphs for all characters in the text, False otherwise.
    """

    font = fontTools.ttLib.TTFont(font_path)
    cmap = font['cmap'].tables[0].ttFont.tables['cmap'].tables[0].cmap

    missing_glyphs = []
    for char in text:
        glyph_index = cmap.get(ord(char))
        if glyph_index is None:
            missing_glyphs.append(char)

    return not missing_glyphs

font_path = rf"D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\latin\Aller_Bd.ttf"  # Replace with the path to your font file
font_path = rf"D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\NotoSansArabic-Regular.ttf" 
text = "عمل"

has_arabic = has_arabic_glyphs(font_path, text)
print(f"Font has Arabic glyphs for 'عمل': {has_arabic}")


Font has Arabic glyphs for 'عمل': True


# detect

In [18]:
import glob, os, shutil
from fontTools.ttLib import TTFont

def check_font_support(font_path, text_to_check):
    try:
        font = TTFont(font_path)
        # Check if the font supports the Unicode code points of the Arabic text
        supported = all(ord(char) in font.getBestCmap() for char in text_to_check)
        
        return supported
    except Exception as e:
        print(f"Error while checking font: {e} {font_path[-40:]}")
        return False

csup = cnsu = 0
files = glob.glob(r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\*')
for font_path in files:
    # arabic_text = "عمل", '_notsupport'
    # arabic_text = "٠١٢٣٤٥٦٧٨٩", '_notsupport'
    # text , movename = "abc123",'_only'
    # text , movename = "123",'_only'
    # text , movename = "؟،",'_not_arsymbol'
    text , movename = "#@!?%&",'_not_ensymbol'
    result = check_font_support(font_path, text)
    if result:
        ''
    else:
        new_folder = os.path.dirname(font_path) + movename
        os.makedirs(new_folder, exist_ok=1)       
        shutil.move(font_path, new_folder)
        cnsu+=1
    
    csup+=result
    

print(csup, cnsu)

468 22


# Draw text on all font files

In [9]:
from PIL import Image, ImageDraw, ImageFont
import glob, os, string
from fontTools.ttLib import TTFont
texts = []
texts += ["٠١٢٣٤٥٦٧٨٩"]
texts += ["0123456789"]
texts += ["أبتثجحخدذرزسشصضطظعغفقكلمنهوي"]
texts += string.ascii_letters.split('A')
texts += ["«»؟،؛٪✓≥≤"+'=><', "!\"#$%&'()*+,-./:;?@[\\]^_`{|}~ "]

# font_dir = r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\*'
font_dir = r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts_combine\**\*'
output_folder = r'D:\Projects\AISoftArt\CoVi\OCR\output_font2'
os.makedirs(output_folder, exist_ok=1)
font0 = ImageFont.truetype('C:/Windows/Fonts/Arial.ttf', 20)  # Adjust font size as needed
for font_path in glob.glob(font_dir, recursive=1):
    try:
        font = ImageFont.truetype(font_path, 36)  # Adjust font size as needed
    except OSError as e:
        print(f"Error loading font {font_path}: {e}")
        continue

    # Create a new image with a suitable size
    img = Image.new('RGB', (800, 500), color='black')  # Adjust dimensions as needed
    draw = ImageDraw.Draw(img)

    # Calculate text width and position it centrally
    # text_width, _ = draw.textsize(text, font=font)
    # left, top, right, bottom = font.getbbox(text)
    # text_width, text_height = right - left, bottom - top
    # text_x = (img.width - text_width/2) // 2
    # # text_x = right // 2
    # # text_x = (img.width) // 2
    # # text_x = ((img.width) // 2 ) -text_width//4
    # text_y = img.height // 2

    # Draw the text with black color
    # draw.text((text_x, text_y), text, font=font, fill=(0, 0, 0))

    # u = int(len(text)/3)
    # for i in range(u):
        # draw.text((text_x, text_height*(i+1)), text[u*i:u*(i+1)], font=font, fill=(0, 0, 0))
    for i,text in enumerate(texts):
        left, top, right, bottom = font.getbbox(text)
        text_width, text_height = right - left, bottom - top
        text_x = (img.width - right) // 2
        text_y = img.height // 2     

        draw.text((text_x, 50*(i+1)), text, font=font, fill='white')

    fontm = TTFont(font_path)
    # arnum8 = fontm.getBestCmap()[ord('٨')]
    arnum8 = None
    ennum8 = fontm.getBestCmap()[ord('8')]

    notsupport = []
    support = []
    for i,t in enumerate(fontm['cmap'].tables):
        try:
            n = t.cmap[ord('٨')]
            # print()
            support+=[n]
        except:
            notsupport+=[i]


    # draw.text((10, text_height*20),f'num8:{arnum8},{ennum8} ,sup:{support} notsupport:{notsupport}' , font=font0, fill=(0, 0, 0))

    # Extract font name from path for filename
    font_name = os.path.basename(font_path)
    fold = font_path.split('\\')[-2]
    output_path = rf"{output_folder}\{fold}_{font_name}.jpg"  # Adjust output format if needed

    # Save the image with a unique filename
    img.save(output_path)
    print(f"Image saved as: {output_path}")

Error loading font D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts_combine\combine: cannot open resource
Error loading font D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts_combine\en: cannot open resource
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_29ltbukrabolditalic_0.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_29ltbukrabold_0.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_29ltbukralight_0.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_29ltbukraregular_0.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_A.Jannat.LT.Bold_1.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_A.Jannat.LT.Regular_1.ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_abdoullah (afaaq).ttf.jpg
Image saved as: D:\Projects\AISoftArt\CoVi\OCR\output_font2\combine_abdoullah (bold)

In [46]:
chr(1640)

'٨'

In [ ]:
ord(1)

In [69]:
import cairocffi as cairo
import glob

fail = 0
files = glob.glob(r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\*')

for font_file in files:
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, 1, 1)
    context = cairo.Context(surface)

    font_face = cairo.ToyFontFace(font_file, 0)
    # font_face = cairo.FontFace(font_file, 0)
    context.set_font_face(font_face)

    # Check for Arabic number glyphs (0x0660 - 0x0669)
    # for i in range(0x0660, 0x066A):
    for i in range(0x0661, 0x0662):     
        # print(i)   
        glyph = context.glyph_path(i)
        # glyph = context.glyph_path(glyphs)
        if glyph is None:
            # print(f"{font_file} does NOT support Arabic number {chr(i)}")
            ''
        else:
            print(f"{font_file} supports Arabic number {chr(i)}")


AttributeError: 'ImageSurface' object has no attribute 'context'

In [31]:
from fontTools.ttLib import TTFont
import random
files = glob.glob(r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\*')
font_path = random.choice(files)
font = TTFont(font_path)
font.getBestCmap()[1640]

'eightarabic'

In [34]:
from fontTools.ttLib import TTFont
font_path = random.choice(files)
font = TTFont(font_path)
cmap = font['cmap']
for table in cmap.tables:
    dir(table)
    break
has_arabic_digits = any(table.is_horizontal() and 0x0660 <= table.firstCode <= 0x0669 for table in cmap.tables)
print(f"Font supports Arabic digits: {has_arabic_digits}")


AttributeError: is_horizontal

In [37]:
is_attributes = [attr for attr in dir(table) if attr.startswith("is")]
print("Attributes starting with 'is*':", is_attributes)

Attributes starting with 'is*': ['isSymbol', 'isUnicode']


In [38]:
from fontTools.ttLib import TTFont

font_path = random.choice(files)
font = TTFont(font_path)
cmap = font['cmap']

has_arabic_digits = False
for table in cmap.tables:
    # Load the cmap table data to access is_horizontal()
    table.ensureDecompiled()

    if table.is_horizontal() and 0x0660 <= table.firstCode <= 0x0669:
        has_arabic_digits = True
        break  # No need to check further tables

print(f"Font supports Arabic digits: {has_arabic_digits}")


AttributeError: is_horizontal

In [97]:
from fontTools.ttLib import TTFont
import random
import glob

# files = glob.glob(r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\Al-Jazeera-Arabic-Bold_0*')
files = glob.glob(r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\*')
# font_path = random.choice(files)
# files = [random.choice(files)]
for font_path in files:
    fontm = TTFont(font_path)
    cmap = fontm['cmap']

    # Check if any CmapSubtable has Arabic digits
    has_arabic_digits = any(
        table.format == 4 and 0x0660 <= code <= 0x0669
        for table in cmap.tables
        for code in table.cmap.keys()
    )
    # if not has_arabic_digits:
    # print(f"Font supports Arabic digits: {has_arabic_digits}")
    # print(table.cmap[1640])
    # print(table.cmap[ord('٨')])
    # print(fontm.getBestCmap()[ord('٨')])
    notsupport = 0
    for t in fontm['cmap'].tables:
        try:
            t.cmap[ord('٨')]
            # print()
        except:
            notsupport+=1
    print('notsupport',notsupport)

notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 0
notsupport 0
notsupport 0
notsupport 0
notsupport 1
notsupport 0
notsupport 0
notsupport 0
notsupport 0
notsupport 0
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 1
notsupport 0
notsupport 1
notsupport 0
notsupport 0

#move lack numar

In [28]:
import shutil
fs=r'D:\Projects\AISoftArt\CoVi\OCR\font_notgoodarnum\*'
src = r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\\'
target  = r'D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar_lacknumar\\'
os.makedirs(target, exist_ok=1)
for f in glob.glob(fs):
    f1=os.path.basename(f)
    f1= os.path.splitext(f1)[0]
    srcf = glob.glob(src+f1+'.*')[0]
    print(f1,srcf)
    shutil.move(srcf, target)

Ahrar Font D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\Ahrar Font.otf
Al Ekbariah font D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\Al Ekbariah font.ttf
AL-Gemah-Badr D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AL-Gemah-Badr.ttf
AL-Gemah-King D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AL-Gemah-King.ttf
AL-Gemah-Sana D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AL-Gemah-Sana.ttf
AL-Gemah-Sana1 D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AL-Gemah-Sana1.ttf
AL3RABIA D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AL3RABIA.TTF
AraHalaBoShe'sha-Regular D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AraHalaBoShe'sha-Regular.otf
AraHalaBoShe_sha-Regular D:\Projects\AISoftArt\CoVi\OCR\TextRecognitionDataGenerator\trdg\fonts\ar\AraHalaBoShe_sha-Regular.otf
AraHamah19